In [ ]:
# check code: https://github.com/bernorieder/SexCams/blob/master/timestamp_calc_all_platforms.ipynb
# churn/regulars/classficiation
# failed scrapes: https://docs.google.com/spreadsheets/d/1Y5yldgDYdvCWYJZX7SevwL5HCcaxzbyo/edit#gid=1279771614

import glob
from pydoc import plain
import pandas as pd
from datetime import datetime

platform = 'myfreecams'

# is room_count on myfreecams viewers? where did the bongacams viewers go? (Emilija will check with Sarah)
var_viewers = {'bongacams':False,'chaturbate':True,'ebonycams':False,'livejasmin':False,'myfreecams':True,'streamate':False}

# read file names from directory
path = 'homepage_csvs/' + platform + '/'
csv_files = glob.glob(path + '*.csv')

print(len(csv_files))

# iterate over files to create a dict of performers and their sessions
performerlist = {}
counter = 0
for filename in sorted(csv_files):
    
    counter += 1
    if counter % 100 == 0:
        print(counter)
    
    try:
        df = pd.read_csv(filename).fillna(0)
        if platform == 'myfreecams':
            filename = filename.replace("cams/myfreecams", "cams/myfreecams_")
            df.rename(columns = {'room_count':'viewers'}, inplace = True)
        fileparts = filename.split('_')
        date_string = fileparts[2] + ' ' + fileparts[3]
        
        date = datetime.strptime(date_string, '%Y-%m-%d %H-%M-%S')
        rows = len(df)
        
        for idx, values in df.iterrows():
            if values['name'] not in performerlist:
                performerlist[values['name']] = dict()
            if(var_viewers[platform]):
                # replace bongacams 9999+ placeholder
                if values['viewers'] == '9999+':
                    values['viewers'] = 10000
                performerlist[values['name']][date] = (values['position'],int(values['viewers']),values['position']/rows)
            else:
                performerlist[values['name']][date] = (values['position'],0,values['position']/rows)
    except:
        print(filename)
            
print(len(performerlist))


In [ ]:
days = {}
for name in performerlist:
    performer = sorted(performerlist[name].items())
    for stamp in performer:
        myday = str(stamp[0].year) + "-" + str(stamp[0].month) + "-" + str(stamp[0].day)
        if name not in days:
            days[name] = {}
        days[name][myday] = True

daylist = {}
for name in days:
    if name not in daylist:
        daylist[name] = len(days[name])

#daylist

In [ ]:
sessions = []
sid = 0
# days active?
# add to performer file

for name in performerlist:
    
    #if name != 'vany_love':
    #   continue
       
    performer = sorted(performerlist[name].items())
    #if name == '':
    #print(name, performer)
    #continue
    
    last_stamp = None
    counter = 0
    
    for stamp in performer:
        
        if last_stamp == None:
            
            ns = {}
            ns['id'] = sid
            sid += 1
            ns['name'] = name
            ns['start'] = stamp[0]
            if(var_viewers[platform]):
                ns['start_viewers'] = stamp[1][1]
            ns['start_position'] = stamp[1][0]
            ns['start_position_norm'] = stamp[1][2]
            ns['positions'] = []
            ns['positions_norm'] = []
            ns['positions'].append(stamp[1][0])
            ns['positions_norm'].append(stamp[1][2])
            if(var_viewers[platform]):
                ns['viewers'] = []
                ns['viewers'].append(stamp[1][1])
            ns['days_active'] = daylist[name]
            
            last_stamp = stamp
        
        else:
            
            delta = stamp[0] - last_stamp[0]
            #print(delta.total_seconds())
            
            if delta.total_seconds() > 3700 and counter < len(performer):
                ns['end'] = last_stamp[0]
                if(var_viewers[platform]):
                    ns['end_viewers'] = last_stamp[1][1]
                ns['end_position'] = last_stamp[1][0]
                ns['end_position_norm'] = last_stamp[1][2]
                ns['duration'] = round((last_stamp[0] - ns['start']).total_seconds() / 60)
                ns['average_position'] = round(sum(ns['positions']) / len(ns['positions']))
                ns['average_position_norm'] = sum(ns['positions_norm']) / len(ns['positions_norm'])
                ns['variance_position'] = round(sum((i - ns['average_position']) ** 2 for i in ns['positions']) / len(ns['positions']))
                ns['variance_position_norm'] = sum((i - ns['average_position_norm']) ** 2 for i in ns['positions_norm']) / len(ns['positions_norm'])
                ns['max_position'] = sorted(ns['positions'])[len(ns['positions'])-1]
                ns['max_position_norm'] = sorted(ns['positions_norm'])[len(ns['positions_norm'])-1]
                ns['min_position'] = sorted(ns['positions'])[0]
                ns['min_position_norm'] = sorted(ns['positions_norm'])[0]
                if(var_viewers[platform]):
                    ns['average_viewers'] = round(sum(ns['viewers']) / len(ns['viewers']))
                    ns['variance_viewers'] = round(sum((i - ns['average_viewers']) ** 2 for i in ns['viewers']) / len(ns['viewers']))
                    ns['max_viewers'] = sorted(ns['viewers'])[len(ns['viewers'])-1]
                    ns['min_viewers'] = sorted(ns['viewers'])[0]
                sessions.append(ns)
                
                ns = {}
                ns['id'] = sid
                sid += 1
                ns['name'] = name
                ns['start'] = stamp[0]
                if(var_viewers[platform]):
                    ns['start_viewers'] = stamp[1][1]
                ns['start_position'] = stamp[1][0]
                ns['start_position_norm'] = stamp[1][2]
                ns['positions'] = []
                ns['positions_norm'] = []
                ns['positions'].append(stamp[1][0])
                ns['positions_norm'].append(stamp[1][2])
                if(var_viewers[platform]):
                    ns['viewers'] = []
                    ns['viewers'].append(stamp[1][1])
                ns['days_active'] = daylist[name]
            
            else:
                
                ns['positions'].append(stamp[1][0])
                ns['positions_norm'].append(stamp[1][2])
                if(var_viewers[platform]):
                    ns['viewers'].append(stamp[1][1])
                
            last_stamp = stamp

        if counter == len(performer)-1:
            ns['end'] = stamp[0]
            if(var_viewers[platform]):
                ns['end_viewers'] = stamp[1][1]
            ns['end_position'] = stamp[1][0]
            ns['end_position_norm'] = stamp[1][2]
            ns['duration'] = round((stamp[0] - ns['start']).total_seconds() / 60)
            ns['average_position'] = round(sum(ns['positions']) / len(ns['positions']))
            ns['average_position_norm'] = sum(ns['positions_norm']) / len(ns['positions_norm'])
            ns['variance_position'] = round(sum((i - ns['average_position']) ** 2 for i in ns['positions']) / len(ns['positions']))
            ns['variance_position_norm'] = sum((i - ns['average_position_norm']) ** 2 for i in ns['positions_norm']) / len(ns['positions_norm'])
            ns['max_position'] = sorted(ns['positions'])[len(ns['positions'])-1]
            ns['max_position_norm'] = sorted(ns['positions_norm'])[len(ns['positions_norm'])-1]
            ns['min_position'] = sorted(ns['positions'])[0]
            ns['min_position_norm'] = sorted(ns['positions_norm'])[0]
            if(var_viewers[platform]):
                ns['average_viewers'] = round(sum(ns['viewers']) / len(ns['viewers']))
                ns['variance_viewers'] = round(sum((i - ns['average_viewers']) ** 2 for i in ns['viewers']) / len(ns['viewers']))
                ns['max_viewers'] = sorted(ns['viewers'])[len(ns['viewers'])-1]
                ns['min_viewers'] = sorted(ns['viewers'])[0]
            sessions.append(ns)
            last_stamp = None
        
        counter += 1
        

df = pd.DataFrame.from_dict(sessions)
df.to_csv('sessions_'+platform+'.csv',index=False)